# Sistema RAG Completo - Llama 3.1 8B

Este notebook combina a construção do índice e a execução do sistema RAG com avaliação de métricas.

## Estrutura:
1. **Configuração e Imports**
2. **Construção do Índice** (executar 1x e salvar)
3. **Carregamento do Sistema RAG** (carregar modelos salvos)
4. **Execução de Perguntas** (rodar N vezes)
5. **Avaliação de Métricas** (análise separada dos resultados)

---
## 1. Configuração e Imports

In [1]:
%%writefile requirements.txt

# --- Índices para pacotes pré-compilados com CUDA 12.4 ---
--extra-index-url https://download.pytorch.org/whl/cu126
--extra-index-url https://abetlen.github.io/llama-cpp-python/whl/cu124

# --- Versões Fixas para Estabilidade ---

# Core de IA e Machine Learning
torch
torchvision
torchaudio
#numpy
#pandas

# Bibliotecas do RAG
langchain
langchain-community
langchain-core
sentence-transformers
faiss-gpu-cu12
llama-cpp-python

# Ferramentas e Utilitários
PyMuPDF
unstructured[pdf]==0.18.15
huggingface_hub
evaluate
rouge_score
tqdm
poppler-utils
bert_score



Writing requirements.txt


In [2]:
!pip install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu126, https://abetlen.github.io/llama-cpp-python/whl/cu124
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 551.3/551.3 MB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 33.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 5.0 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of opencv-python to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 74.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 47.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.1/48.1 MB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 68.6 MB/s eta 0:00

In [1]:
import os
import shutil
import torch
import pickle
import re
import gc
import pandas as pd
import pymupdf
from tqdm import tqdm
from langchain_core.documents import Document
from langchain.storage import InMemoryStore
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.retrievers import ParentDocumentRetriever
from langchain.prompts import PromptTemplate
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain_community.llms import LlamaCpp
from langchain.chains import LLMChain
from huggingface_hub import hf_hub_download
from sentence_transformers import CrossEncoder
import evaluate

print("✓ Imports realizados com sucesso!")
print(f"Device disponível: {'CUDA' if torch.cuda.is_available() else 'CPU'}")

✓ Imports realizados com sucesso!
Device disponível: CUDA


In [3]:
# PARÂMETROS GLOBAIS
DOCS_PATH = "documentos/"
VECTORSTORE_PATH = "faiss_final_index"
DOCSTORE_PATH = "final_docstore"
MODEL_EMBEDDING_NAME = "BAAI/bge-m3"
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# Configuração do Modelo Llama 3.1
MODEL_REPO_ID = "lmstudio-community/Meta-Llama-3.1-8B-Instruct-GGUF"
MODEL_BASENAME = "Meta-Llama-3.1-8B-Instruct-Q5_K_M.gguf"
LLM_N_GPU_LAYERS = -1
LLM_N_CTX = 8192
RETRIEVER_K = 10
TOP_K_AFTER_RERANK = 4
CROSS_ENCODER_MODEL = 'cross-encoder/ms-marco-MiniLM-L-6-v2'

# verificar e criar pasta DOCS_PATH
if not os.path.exists(DOCS_PATH):
    os.makedirs(DOCS_PATH)
    print(f"Pasta '{DOCS_PATH}' criada com sucesso.")
else:
    print(f"A pasta '{DOCS_PATH}' já existe.")

print("✓ Parâmetros configurados!")


A pasta 'documentos/' já existe.
✓ Parâmetros configurados!


---
## 2. Construção do Índice FAISS
**⚠️ Execute esta seção apenas UMA VEZ ou quando atualizar os documentos**

In [10]:
def cleanup_old_index(vectorstore_path, docstore_path):
    """Limpa os diretórios antigos do índice e do docstore."""
    for path in [vectorstore_path, docstore_path]:
        if os.path.exists(path):
            shutil.rmtree(path)
    os.makedirs(vectorstore_path, exist_ok=True)
    os.makedirs(docstore_path, exist_ok=True)
    print("Limpeza e recriação das pastas concluída.")

def load_and_clean_definitively(folder_path: str) -> list:
    """
    Solução definitiva: Extrai blocos de texto, ignora cabeçalhos/rodapés,
    ordena pela ordem de leitura, une o conteúdo e normaliza os espaços.
    """
    pdf_files = [f for f in os.listdir(folder_path) if f.endswith(".pdf")]
    all_cleaned_pages = []

    for filename in tqdm(pdf_files, desc="Processando PDFs de forma definitiva"):
        file_path = os.path.join(folder_path, filename)

        with pymupdf.open(file_path) as doc:
            for page_num, page in enumerate(doc):
                page_height = page.rect.height
                margin_top = page_height * 0.08
                margin_bottom = page_height * 0.92

                blocks = page.get_text("blocks")
                valid_blocks = [b for b in blocks if margin_top < b[1] < margin_bottom]
                valid_blocks.sort(key=lambda b: b[1])

                page_text = " ".join([b[4].replace('\n', ' ') for b in valid_blocks])
                all_cleaned_pages.append(page_text)

    full_text_content = " ".join(all_cleaned_pages)
    full_text_content = re.sub(r'\s+', ' ', full_text_content).strip()

    print(f"Limpeza concluída. Conteúdo total com {len(full_text_content)} caracteres.")
    return [Document(page_content=full_text_content)]

print("✓ Funções de processamento definidas!")

✓ Funções de processamento definidas!


In [11]:
# EXECUTAR APENAS QUANDO NECESSÁRIO RECONSTRUIR O ÍNDICE
print("Iniciando construção do índice...\n")

cleanup_old_index(VECTORSTORE_PATH, DOCSTORE_PATH)
docs = load_and_clean_definitively(DOCS_PATH)

# Splitters para a estratégia Parent/Child
parent_splitter = RecursiveCharacterTextSplitter(chunk_size=1500, chunk_overlap=200, add_start_index=True)
child_splitter = RecursiveCharacterTextSplitter(chunk_size=400, chunk_overlap=50)

embeddings = HuggingFaceEmbeddings(model_name=MODEL_EMBEDDING_NAME, model_kwargs={'device': DEVICE})

vectorstore = FAISS.from_texts(texts=["_INITIALIZING_"], embedding=embeddings)
vectorstore.delete(list(vectorstore.index_to_docstore_id.values()))

store = InMemoryStore()

retriever = ParentDocumentRetriever(
    vectorstore=vectorstore, docstore=store,
    child_splitter=child_splitter, parent_splitter=parent_splitter,
)

print("Adicionando documento limpo ao índice...")
retriever.add_documents(docs, ids=None)

vectorstore.save_local(VECTORSTORE_PATH)
with open(os.path.join(DOCSTORE_PATH, "store.pkl"), "wb") as f:
    pickle.dump(store, f)

print(f"\n✓ Índice definitivo salvo em '{VECTORSTORE_PATH}' e '{DOCSTORE_PATH}'")

Iniciando construção do índice...

Limpeza e recriação das pastas concluída.


Processando PDFs de forma definitiva: 100%|██████████| 2/2 [00:00<00:00,  2.86it/s]


Limpeza concluída. Conteúdo total com 319880 caracteres.
Adicionando documento limpo ao índice...

✓ Índice definitivo salvo em 'faiss_final_index' e 'final_docstore'


In [9]:
def save_faiss_index_content(retriever, output_file="faiss_index_content.txt"):
    """
    Recupera todos os documentos do índice FAISS (através do retriever)
    e salva o conteúdo em um arquivo de texto estruturado.
    """
    print(f"Salvando conteúdo do índice FAISS em '{output_file}'...")
    try:
        # Como o retriever usa o docstore, podemos iterar sobre ele.
        # No entanto, não há um método direto para obter TODOS os documentos.
        # Uma abordagem é recuperar documentos para uma query genérica e depois
        # tentar recuperar todos os documentos únicos do docstore.
        # Uma forma mais robusta é acessar diretamente o docstore se ele for InMemoryStore

        if isinstance(retriever.docstore, InMemoryStore):
            all_docs = list(retriever.docstore.yield_keys())
            # Para cada key (ID do documento pai), recuperar o documento pai
            # e seus filhos, se houver.
            with open(output_file, "w", encoding="utf-8") as f:
                for doc_id in tqdm(all_docs, desc="Escrevendo documentos no arquivo"):
                    doc = retriever.docstore.mget([doc_id])[0] # Recupera o documento pai
                    f.write(f"--- Document ID: {doc_id} ---\n")
                    f.write(f"Content:\n{doc.page_content}\n")
                    # Adicionar metadados se existirem
                    if doc.metadata:
                        f.write("Metadata:\n")
                        for key, value in doc.metadata.items():
                            f.write(f"  {key}: {value}\n")
                    f.write("-" * 20 + "\n\n")
            print("\n✓ Conteúdo do índice FAISS salvo com sucesso!")
        else:
            print("Erro: O docstore não é InMemoryStore. Não é possível salvar o conteúdo diretamente.")

    except Exception as e:
        print(f"Erro ao salvar o conteúdo do índice FAISS: {e}")

# Exemplo de uso (descomente para rodar):
save_faiss_index_content(retriever)

Salvando conteúdo do índice FAISS em 'faiss_index_content.txt'...


Escrevendo documentos no arquivo: 100%|██████████| 246/246 [00:00<00:00, 88293.58it/s]


✓ Conteúdo do índice FAISS salvo com sucesso!


---
## 3. Carregamento do Sistema RAG
**Execute esta seção para carregar os índices e modelos já salvos**

In [12]:
# Carregar embeddings
print("Carregando modelo de embeddings...")
embeddings = HuggingFaceEmbeddings(
    model_name=MODEL_EMBEDDING_NAME,
    model_kwargs={'device': DEVICE}
)
print("✓ Embeddings carregados!")

Carregando modelo de embeddings...
✓ Embeddings carregados!


In [13]:
# Carregar vectorstore FAISS
print("Carregando índice FAISS...")
vectorstore = FAISS.load_local(
    VECTORSTORE_PATH,
    embeddings,
    allow_dangerous_deserialization=True
)
print("✓ Vectorstore carregado!")

Carregando índice FAISS...
✓ Vectorstore carregado!


In [14]:
# Carregar docstore
print("Carregando docstore...")
with open(os.path.join(DOCSTORE_PATH, "store.pkl"), "rb") as f:
    store = pickle.load(f)

child_splitter = RecursiveCharacterTextSplitter(chunk_size=400, chunk_overlap=50)
retriever = ParentDocumentRetriever(
    vectorstore=vectorstore,
    docstore=store,
    child_splitter=child_splitter
)
retriever.search_kwargs['k'] = RETRIEVER_K
print("✓ Retriever configurado!")

Carregando docstore...
✓ Retriever configurado!


In [15]:
# Carregar cross-encoder
print("Carregando cross-encoder...")
cross_encoder = CrossEncoder(CROSS_ENCODER_MODEL)
print("✓ Cross-encoder carregado!")

Carregando cross-encoder...


config.json:   0%|          | 0.00/794 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

✓ Cross-encoder carregado!


In [16]:
# Baixar e carregar LLM
print(f"Baixando modelo: {MODEL_BASENAME}...")
model_path = hf_hub_download(repo_id=MODEL_REPO_ID, filename=MODEL_BASENAME)
print(f"✓ Modelo baixado: {model_path}")

print("\nInicializando LLM...")
llm = LlamaCpp(
    model_path=model_path,
    n_gpu_layers=LLM_N_GPU_LAYERS,
    n_ctx=LLM_N_CTX,
    max_tokens=512,
    repeat_penalty=1.15,
    temperature=0.1,
    top_p=0.9,
    top_k=40,
    n_batch=512,
    f16_kv=True,
    callbacks=[StreamingStdOutCallbackHandler()],
    verbose=False
)
print("✓ LLM carregado!")

Baixando modelo: Meta-Llama-3.1-8B-Instruct-Q5_K_M.gguf...


Meta-Llama-3.1-8B-Instruct-Q5_K_M.gguf:   0%|          | 0.00/5.73G [00:00<?, ?B/s]

✓ Modelo baixado: /root/.cache/huggingface/hub/models--lmstudio-community--Meta-Llama-3.1-8B-Instruct-GGUF/snapshots/8601e6db71269a2b12255ebdf09ab75becf22cc8/Meta-Llama-3.1-8B-Instruct-Q5_K_M.gguf

Inicializando LLM...


llama_context: n_ctx_per_seq (8192) < n_ctx_train (131072) -- the full capacity of the model will not be utilized


✓ LLM carregado!


In [63]:
# Criar chain RAG
template = (
    "<|start_header_id|>system<|end_header_id|>\n\n"
    "Você é um assistente especializado em responder perguntas com base em documentos acadêmicos.\n\n"
    "INSTRUÇÕES:\n"
    "- Leia atentamente o contexto fornecido\n"
    "- Responda de forma completa e natural, como em uma conversa\n"
    "- Use APENAS informações presentes no contexto\n"
    "- Para perguntas sobre 'quando', forneça a resposta em uma frase completa mencionando o ano e o local\n"
    "- Seja preciso mas não robotizado\n"
    "- Evite falar 'De acordo com o contexto fornecido'\n"
    "- Evite respostas de uma única palavra ou número<|eot_id|>"
    "<|start_header_id|>user<|end_header_id|>\n\n"
    "Contexto:\n{context}\n\n"
    "Pergunta: {question}<|eot_id|>"
    "<|start_header_id|>assistant<|end_header_id|>\n\n"
)

prompt = PromptTemplate(template=template, input_variables=["context", "question"])
rag_chain = LLMChain(prompt=prompt, llm=llm)

print("✓ RAG Chain criada!")
print("\n" + "="*80)
print("SISTEMA RAG PRONTO PARA USO!")
print("="*80)

✓ RAG Chain criada!

SISTEMA RAG PRONTO PARA USO!


---
## 4. Funções de Execução de Perguntas

In [64]:
def rerank_documents(question, retrieved_docs, cross_encoder, top_k):
    """Reordena os documentos usando um CrossEncoder."""
    pairs = [[question, doc.page_content] for doc in retrieved_docs]
    scores = cross_encoder.predict(pairs)
    doc_scores = list(zip(retrieved_docs, scores))
    doc_scores_sorted = sorted(doc_scores, key=lambda x: x[1], reverse=True)

    print(f"\n[DEBUG] Top {top_k} documentos após re-ranking:")
    for i, (doc, score) in enumerate(doc_scores_sorted[:top_k], 1):
        preview = doc.page_content[:80].replace('\n', ' ')
        print(f"  #{i} (score: {score:.4f}): {preview}...")

    return [doc for doc, score in doc_scores_sorted[:top_k]]

def ask(question, rag_chain, retriever, cross_encoder, verbose=True):
    """Pipeline completo: Retrieve -> Re-rank -> Generate."""
    if verbose:
        print(f"\n{'='*80}")
        print(f"PERGUNTA: {question}")
        print('='*80)

    retrieved_docs = retriever.invoke(question)
    if verbose:
        print(f"✓ Recuperados {len(retrieved_docs)} documentos")

    reranked_docs = rerank_documents(
        question, retrieved_docs, cross_encoder, TOP_K_AFTER_RERANK)

    context_parts = []
    for i, doc in enumerate(reranked_docs, 1):
        context_parts.append(f"[Trecho {i}]: {doc.page_content}")

    context = "\n\n".join(context_parts)

    if verbose:
        print(f"\n[DEBUG] Primeiros 400 caracteres do contexto:")
        print("-" * 80)
        print(context[:400])
        print("-" * 80)

    result = rag_chain.invoke({"context": context, "question": question})
    answer = result['text'].strip()

    # Limpeza de tokens especiais do Llama 3.1
    tokens_to_remove = ["</s>", "<|eot_id|>", "<|end_header_id|>", "<|start_header_id|>"]
    for token in tokens_to_remove:
        answer = answer.replace(token, "")

    answer = answer.strip()

    if verbose:
        print(f"\n{'='*80}")
        print(f"RESPOSTA: {answer}")
        print('='*80)

    return answer, context

print("✓ Funções de execução definidas!")

✓ Funções de execução definidas!


---
## 5. Dataset de Avaliação
**Adicione ou modifique as perguntas aqui**

In [65]:
# Dataset de perguntas e respostas esperadas
evaluation_dataset = [
    {
        "pergunta": "Quando o curso de Tecnologia em Análise e Desenvolvimento de Sistemas foi implantado no Campus de Picos?",
        "resposta_esperada": "O curso foi implantado no Campus de Picos em 2013."
    },
    {
        "pergunta": "Quais são as modalidades aceitas para a apresentação do TCC?",
        "resposta_esperada": "O TCC pode ser apresentado em formato de Monografia, Artigo Científico, Relatório Técnico de Software (RTS) ou Relatório Técnico de Trabalho/Estágio (RTT)."
    },
    {
        "pergunta": "Qual foi o montante total de investimento que o setor de TI no Brasil atingiu em 2021, englobando os mercados de software, serviços, hardware e exportações?",
        "resposta_esperada": "O investimento atingiu R$238,2 bilhões (US$ 46,2 bilhões)."
    },
    {
        "pergunta": "Qual o pré-requisito para cursar Programação para Dispositivos Móveis?",
        "resposta_esperada": "O pré-requisito para cursar Programação para Dispositivos Móveis é a disciplina de Programação Orientada a Objetos."
    },
    {
        "pergunta": "Quantas vagas anuais são ofertadas para o curso de ADS no campus Picos e qual o turno de funcionamento?",
        "resposta_esperada": "São ofertadas 40 vagas por ano para o turno vespertino."
    },

    # novas perguntas

    {
        "pergunta": "Qual é a carga horária total do curso de ADS e como ela é dividida entre disciplinas obrigatórias e atividades complementares?",
        "resposta_esperada": "A carga horária total do curso é de 2100 horas, distribuídas em 2000 horas de disciplinas obrigatórias e 100 horas de atividades complementares."
    },
    {
        "pergunta": "Quais são os dois laboratórios de pesquisa e extensão disponíveis para os alunos do curso de ADS desenvolverem projetos?",
        "resposta_esperada": "Os alunos podem desenvolver projetos de pesquisa e extensão nos laboratórios Mambee (Fábrica Escola de Software) e LIARA (Laboratório de Inteligência Artificial, Robótica e Automação)."
    },
    {
        "pergunta": "Como é calculada a nota final do Trabalho de Conclusão de Curso (TCC)?",
        "resposta_esperada": "A nota final do TCC é calculada por uma média ponderada, onde 50% vem da nota do orientador e os outros 50% são divididos igualmente entre os outros dois membros da banca examinadora."
    },
    {
        "pergunta": "Qual disciplina é o pré-requisito necessário para cursar Redes de Computadores?",
        "resposta_esperada": "O pré-requisito para cursar a disciplina de Redes de Computadores é a disciplina de Introdução a Computação."
    },
    {
        "pergunta": "Em que ano a instituição, que hoje é o IFPI, foi efetivamente transformada em Centro Federal de Educação Tecnológica do Piauí (CEFET-PI)?",
        "resposta_esperada": "A transformação da instituição em Centro Federal de Educação Tecnológica do Piauí (CEFET-PI) foi efetivada em 1999."
    },

    # # organização didática

    # {
    #     "pergunta": "Quais modalidades de curso permitem o trancamento de matrícula no IFPI?",
    #     "resposta_esperada": "O trancamento de matrícula é restrito aos cursos superiores e técnicos concomitantes/subsequentes."
    # },
    # {
    #     "pergunta": "Qual a média semestral mínima necessária para que um aluno de curso superior de graduação seja aprovado diretamente em uma disciplina, sem precisar fazer o Exame Final?",
    #     "resposta_esperada": "É necessária uma média igual ou superior a 7,0 (sete)."
    # },
    # {
    #     "pergunta": "Qual é a frequência mínima que um aluno dos cursos técnicos integrados precisa ter para ser aprovado em uma disciplina?",
    #     "resposta_esperada": "A frequência deve ser igual ou superior a 75% da carga horária total da disciplina."
    # },
    # {
    #     "pergunta": "A reprovação em um módulo por quantas vezes consecutivas pode levar ao cancelamento compulsório da matrícula de um estudante?",
    #     "resposta_esperada": "A reprovação em séries/módulos por três vezes consecutivas pode levar ao cancelamento."
    # },
    # {
    #     "pergunta": "Além de ministrar aulas, qual é um dos direitos assegurados ao corpo docente do IFPI relacionado à organização dos cursos?",
    #     "resposta_esperada": "É direito do docente participar da elaboração e execução de projetos, planos de cursos e outras atividades didáticas."
    # },
    # {
    #     "pergunta": "O que é vedado ao corpo discente em relação a bebidas nas dependências da instituição?",
    #     "resposta_esperada": "É vedado portar ou ingerir bebidas alcoólicas."
    # },
    # {
    #     "pergunta": "Qual o prazo máximo para um aluno requerer a segunda chamada de uma avaliação que não pôde comparecer?",
    #     "resposta_esperada": "O prazo é de até 72 horas (setenta e duas horas), considerando os dias úteis, após a data da avaliação."
    # },
    # {
    #     "pergunta": "De que maneiras a educação profissional técnica de nível médio pode ser ofertada em articulação com o ensino médio?",
    #     "resposta_esperada": "Pode ser ofertada de forma integrada, concomitante ou subsequente."
    # },
    # {
    #     "pergunta": "Qual é a penalidade aplicável a um aluno que comete agressão física contra outra pessoa dentro do campus?",
    #     "resposta_esperada": "A penalidade é o cancelamento da matrícula."
    # },
    # {
    #     "pergunta": "Para que um estudante consiga o aproveitamento de estudos em uma disciplina, qual o percentual mínimo de compatibilidade de conteúdo e carga horária exigido?",
    #     "resposta_esperada": "É exigida uma compatibilidade de pelo menos 75%."
    # }
]

print(f"✓ Dataset com {len(evaluation_dataset)} perguntas carregado!")

✓ Dataset com 10 perguntas carregado!


---
## 6. Execução das Perguntas
**Execute esta célula para processar todas as perguntas do dataset**

In [66]:
# Armazenar resultados
results = []

for idx, item in enumerate(evaluation_dataset, 1):
    print(f"\n\n{'#'*80}")
    print(f"PROCESSANDO PERGUNTA {idx}/{len(evaluation_dataset)}")
    print(f"{'#'*80}")

    generated_answer, retrieved_context = ask(
        item["pergunta"],
        rag_chain,
        retriever,
        cross_encoder,
        verbose=True
    )

    results.append({
        "pergunta": item["pergunta"],
        "resposta_esperada": item["resposta_esperada"],
        "resposta_gerada": generated_answer,
        "contexto_recuperado": retrieved_context
    })

# Criar DataFrame com os resultados
df_results = pd.DataFrame(results)

print("\n\n" + "="*80)
print("✓ TODAS AS PERGUNTAS PROCESSADAS!")
print("="*80)
print(f"Total de perguntas: {len(results)}")



################################################################################
PROCESSANDO PERGUNTA 1/10
################################################################################

PERGUNTA: Quando o curso de Tecnologia em Análise e Desenvolvimento de Sistemas foi implantado no Campus de Picos?
✓ Recuperados 9 documentos

[DEBUG] Top 4 documentos após re-ranking:
  #1 (score: 5.8266): no campus Teresina Central. O primeiro vestibular para esse curso ocorreu em jul...
  #2 (score: 4.3149): de Tecnologia - CNCST (documento que elenca as denominações e respectivos descri...
  #3 (score: 1.9619): a inauguração da primeira Unidade de Ensino Descentralizada (UNED) na cidade de ...
  #4 (score: 1.4754): Chamadas Públicas Caso não ocorra o preenchimento de todas as vagas ofertadas no...

[DEBUG] Primeiros 400 caracteres do contexto:
--------------------------------------------------------------------------------
[Trecho 1]: no campus Teresina Central. O primeiro vestibular para esse 

In [ ]:
# Visualizar resultados
print("\nPRÉVIA DOS RESULTADOS:\n")
for idx, row in df_results.iterrows():
    print(f"\n{'='*80}")
    print(f"PERGUNTA {idx+1}:")
    print(f"  {row['pergunta']}")
    print(f"\nESPERADA:")
    print(f"  {row['resposta_esperada']}")
    print(f"\nGERADA:")
    print(f"  {row['resposta_gerada']}")
    print("="*80)


PRÉVIA DOS RESULTADOS:


PERGUNTA 1:
  Quando o curso de Tecnologia em Análise e Desenvolvimento de Sistemas foi implantado no Campus de Picos?

ESPERADA:
  O curso foi implantado no Campus de Picos em 2013.

GERADA:
  Em 2013, no Campus de Picos.

PERGUNTA 2:
  Quais são as modalidades aceitas para a apresentação do TCC?

ESPERADA:
  O TCC pode ser apresentado em formato de Monografia, Artigo Científico, Relatório Técnico de Software (RTS) ou Relatório Técnico de Trabalho/Estágio (RTT).

GERADA:
  De acordo com o contexto fornecido, as modalidades aceitas para a apresentação do TCC são:

1. Monografia;
2. Artigo Científico completo (com resultados);
3. Relatório Técnico de Software (RTS); e
4. Relatório Técnico de Trabalho/Estágio (RTT).

PERGUNTA 3:
  Qual foi o montante total de investimento que o setor de TI no Brasil atingiu em 2021, englobando os mercados de software, serviços, hardware e exportações?

ESPERADA:
  O investimento atingiu R$238,2 bilhões (US$ 46,2 bilhões).

GERAD

---
## 7. Avaliação de Métricas
**Execute esta seção após processar as perguntas para calcular as métricas**

In [67]:
# Preparar dados para métricas
generated_answers = df_results["resposta_gerada"].tolist()
expected_answers = df_results["resposta_esperada"].tolist()

print(f"Preparando avaliação de {len(generated_answers)} respostas...")

Preparando avaliação de 10 respostas...


In [68]:
# Carregar métricas
print("Carregando métricas de avaliação...\n")

rouge_metric = evaluate.load('rouge')
bleu_metric = evaluate.load('bleu')
bertscore_metric = evaluate.load('bertscore')

print("✓ Métricas carregadas!")

Carregando métricas de avaliação...

✓ Métricas carregadas!


In [69]:
# Calcular ROUGE
print("Calculando ROUGE...")
rouge_scores = rouge_metric.compute(
    predictions=generated_answers,
    references=expected_answers
)
print("✓ ROUGE calculado!")

Calculando ROUGE...
✓ ROUGE calculado!


In [70]:
# Calcular BLEU
print("Calculando BLEU...")
bleu_scores = bleu_metric.compute(
    predictions=generated_answers,
    references=expected_answers
)
print("✓ BLEU calculado!")

Calculando BLEU...
✓ BLEU calculado!


In [71]:
# Calcular BERTScore (pode demorar)
print("Calculando BERTScore... (pode demorar)")
bertscore_scores = bertscore_metric.compute(
    predictions=generated_answers,
    references=expected_answers,
    lang='pt',
    model_type='distilbert-base-multilingual-cased'
)
print("✓ BERTScore calculado!")

Calculando BERTScore... (pode demorar)
✓ BERTScore calculado!


In [72]:
# Calcular F1-Score
def calculate_f1_token(prediction, reference):
    pred_tokens = set(prediction.lower().split())
    ref_tokens = set(reference.lower().split())

    if len(pred_tokens) == 0 or len(ref_tokens) == 0:
        return 0.0

    common = pred_tokens.intersection(ref_tokens)
    precision = len(common) / len(pred_tokens)
    recall = len(common) / len(ref_tokens)

    if precision + recall == 0:
        return 0.0

    return 2 * (precision * recall) / (precision + recall)

print("Calculando F1-Score...")
f1_scores = [calculate_f1_token(p, r) for p, r in zip(generated_answers, expected_answers)]

avg_f1 = sum(f1_scores) / len(f1_scores)
avg_bertscore = sum(bertscore_scores['f1']) / len(bertscore_scores['f1'])

print("✓ F1-Scorecalculado!")

# Adicionar métricas ao DataFrame
df_results['f1_score'] = f1_scores

Calculando F1-Score...
✓ F1-Scorecalculado!


### 7.1 Visualização dos Resultados das Métricas

In [74]:
# Exibir resultados completos
print("\n" + "="*80)
print("RESULTADOS COMPLETOS DAS MÉTRICAS")
print("="*80)

print("\n[ROUGE - Overlap de N-gramas]")
print(f"  ROUGE-1: {rouge_scores['rouge1']:.4f}")
print(f"  ROUGE-2: {rouge_scores['rouge2']:.4f}")
print(f"  ROUGE-L: {rouge_scores['rougeL']:.4f}")

print("\n[BLEU - Precisão de N-gramas]")
print(f"  BLEU Score: {bleu_scores['bleu']:.4f}")

print("\n[BERTScore - Similaridade Semântica]")
print(f"  F1 (média): {avg_bertscore:.4f}")
print(f"  Precision:  {sum(bertscore_scores['precision'])/len(bertscore_scores['precision']):.4f}")
print(f"  Recall:     {sum(bertscore_scores['recall'])/len(bertscore_scores['recall']):.4f}")

print("\n[F1-Score Token-based]")
print(f"  F1 (média): {avg_f1:.4f}")

print("\n" + "="*80)


RESULTADOS COMPLETOS DAS MÉTRICAS

[ROUGE - Overlap de N-gramas]
  ROUGE-1: 0.6134
  ROUGE-2: 0.4475
  ROUGE-L: 0.5194

[BLEU - Precisão de N-gramas]
  BLEU Score: 0.2327

[BERTScore - Similaridade Semântica]
  F1 (média): 0.9227
  Precision:  0.9002
  Recall:     0.9470

[F1-Score Token-based]
  F1 (média): 0.5532



In [57]:
# Exibir métricas por pergunta e métricas gerais em tabela
print("\nMÉTRICAS DETALHADAS E POR PERGUNTA:\n")

# Criar DataFrame com as métricas gerais
metrics_data = {
    'Métrica': ['ROUGE-1', 'ROUGE-2', 'ROUGE-L', 'BLEU', 'BERTScore (F1 Médio)', 'F1-Score (Token Médio)'],
    'Valor': [
        rouge_scores['rouge1'],
        rouge_scores['rouge2'],
        rouge_scores['rougeL'],
        bleu_scores['bleu'],
        avg_bertscore,
        avg_f1,
    ]
}
df_metrics_general = pd.DataFrame(metrics_data)
print("\nMétricas Gerais:\n")
display(df_metrics_general.round(4))

# Adicionar BERTScore F1 por pergunta ao DataFrame de resultados
df_results['bertscore_f1'] = bertscore_scores['f1']
df_results['bertscore_precision'] = bertscore_scores['precision']
df_results['bertscore_recall'] = bertscore_scores['recall']


# Exibir métricas por pergunta
print("\nMétricas por Pergunta:\n")
display(df_results[['pergunta', 'f1_score', 'bertscore_f1', 'bertscore_precision', 'bertscore_recall']].round(4))


MÉTRICAS DETALHADAS E POR PERGUNTA:


Métricas Gerais:



,Métrica,Valor
0,ROUGE-1,0.4518
1,ROUGE-2,0.3253
2,ROUGE-L,0.4058
3,BLEU,0.1598
4,BERTScore (F1 Médio),0.8942
5,F1-Score (Token Médio),0.4410



Métricas por Pergunta:



,pergunta,f1_score,bertscore_f1,bertscore_precision,bertscore_recall
0,Quando o curso de Tecnologia em Análise e Dese...,0.7200,0.9359,0.8892,0.9878
1,Quais são as modalidades aceitas para a aprese...,0.5789,0.9552,0.9442,0.9665
2,Qual foi o montante total de investimento que ...,0.3333,0.9213,0.8957,0.9483
3,Qual o pré-requisito para cursar Programação p...,0.8148,0.9797,0.9692,0.9906
4,Quantas vagas anuais são ofertadas para o curs...,0.1935,0.8295,0.8046,0.8560
5,Qual é a carga horária total do curso de ADS e...,0.5075,0.8840,0.8364,0.9374
6,Quais são os dois laboratórios de pesquisa e e...,0.7660,0.9544,0.9374,0.9721
7,Como é calculada a nota final do Trabalho de C...,0.3226,0.8401,0.8044,0.8790
8,Qual disciplina é o pré-requisito necessário p...,0.7200,0.9583,0.9562,0.9604
9,"Em que ano a instituição, que hoje é o IFPI, f...",0.6316,0.9516,0.9391,0.9643


### 7.2 Análise Detalhada (Opcional)

In [58]:
# Análise detalhada de cada resposta
print("\nANÁLISE DETALHADA DAS RESPOSTAS:\n")

for idx, row in df_results.iterrows():
    print(f"\n{'='*80}")
    print(f"PERGUNTA {idx+1}:")
    print(f"  {row['pergunta']}")
    print(f"\nRESPOSTA ESPERADA:")
    print(f"  {row['resposta_esperada']}")
    print(f"\nRESPOSTA GERADA:")
    print(f"  {row['resposta_gerada']}")
    print(f"\nMÉTRICAS:")
    print(f"  F1-Score: {row['f1_score']:.4f}")
    print(f"  BERTScore (F1): {row['bertscore_f1']:.4f}")
    print(f"  BERTScore (Precision): {row['bertscore_precision']:.4f}")
    print(f"  BERTScore (Recall): {row['bertscore_recall']:.4f}")
    print(f"  ROUGE-1: {rouge_scores['rouge1']:.4f}")
    print(f"  ROUGE-2: {rouge_scores['rouge2']:.4f}")
    print(f"  ROUGE-L: {rouge_scores['rougeL']:.4f}")
    print(f"  BLEU: {bleu_scores['bleu']:.4f}")
    print("="*80)


ANÁLISE DETALHADA DAS RESPOSTAS:


PERGUNTA 1:
  Quando o curso de Tecnologia em Análise e Desenvolvimento de Sistemas foi implantado no Campus de Picos?

RESPOSTA ESPERADA:
  O curso foi implantado no Campus de Picos em 2013.

RESPOSTA GERADA:
  O curso de Tecnologia em Análise e Desenvolvimento de Sistemas foi implantado no Campus de Picos, em 2013.

MÉTRICAS:
  F1-Score: 0.7200
  BERTScore (F1): 0.9359
  BERTScore (Precision): 0.8892
  BERTScore (Recall): 0.9878
  ROUGE-1: 0.4518
  ROUGE-2: 0.3253
  ROUGE-L: 0.4058
  BLEU: 0.1598

PERGUNTA 2:
  Quais são as modalidades aceitas para a apresentação do TCC?

RESPOSTA ESPERADA:
  O TCC pode ser apresentado em formato de Monografia, Artigo Científico, Relatório Técnico de Software (RTS) ou Relatório Técnico de Trabalho/Estágio (RTT).

RESPOSTA GERADA:
  As modalidades aceitas para a apresentação do TCC são: Monografia, Artigo Científico, Relatório Técnico de Software (RTS) e Relatório Técnico de Trabalho/Estágio (RTT).

MÉTRICAS:
  F1-S

---
## 8. Salvamento dos Resultados

In [59]:
# Salvar resultados em CSV
output_filename = "resultados_rag_llama3.csv"
df_results.to_csv(output_filename, index=False)
print(f"✓ Resultados salvos em '{output_filename}'")

✓ Resultados salvos em 'resultados_rag_llama3.csv'


In [60]:
# Salvar resumo das métricas
metrics_summary = {
    'ROUGE-1': rouge_scores['rouge1'],
    'ROUGE-2': rouge_scores['rouge2'],
    'ROUGE-L': rouge_scores['rougeL'],
    'BLEU': bleu_scores['bleu'],
    'BERTScore_F1': avg_bertscore,
    'BERTScore_Precision': sum(bertscore_scores['precision'])/len(bertscore_scores['precision']),
    'BERTScore_Recall': sum(bertscore_scores['recall'])/len(bertscore_scores['recall']),
    'F1_Token': avg_f1,
    'Total_Perguntas': len(df_results)
}

df_summary = pd.DataFrame([metrics_summary])
summary_filename = "metricas_resumo.csv"
df_summary.to_csv(summary_filename, index=False)
print(f"✓ Resumo das métricas salvo em '{summary_filename}'")

✓ Resumo das métricas salvo em 'metricas_resumo.csv'


---
## 9. Teste Rápido (Pergunta Única)
**Use esta célula para testar uma pergunta específica sem avaliar métricas**

In [ ]:
# Teste rápido com uma pergunta
pergunta_teste = "Qual o pré-requisito para cursar Programação para Dispositivos Móveis?"

resposta, contexto = ask(
    pergunta_teste,
    rag_chain,
    retriever,
    cross_encoder,
    verbose=True
)


PERGUNTA: Qual o pré-requisito para cursar Programação para Dispositivos Móveis?
✓ Recuperados 10 documentos

[DEBUG] Top 4 documentos após re-ranking:
  #1 (score: 2.3419): complementar: 1. WASLAWICK, Raul Sidnei. Introdução a algoritmos e programação c...
  #2 (score: -2.4429): Pré-requisito: Não possui Competência e Habilidades: Apresentação de temas atuai...
  #3 (score: -3.9853): desenvolvimento para a Internet no desenvolvimento de softwares Dinâmicos para a...
  #4 (score: -5.0021): Tulio. Engenharia de Software Moderna: Princípios e Práticas para Desenvolviment...

[DEBUG] Primeiros 400 caracteres do contexto:
--------------------------------------------------------------------------------
[Trecho 1]: complementar: 1. WASLAWICK, Raul Sidnei. Introdução a algoritmos e programação com Python: uma abordagem dirigida por testes. Rio de Janeiro: Elsevier, 2018. 2. PRESSMAN, Roger S.; MAXIM, Bruce R. Engenharia de software: uma abordagem profissional. 8. ed. São Paulo: McGraw-Hill, 

/usr/local/lib/python3.12/dist-packages/llama_cpp/llama.py:1242: RuntimeWarning: Detected duplicate leading "<|begin_of_text|>" in prompt, this will likely reduce response quality, consider removing it...
  warnings.warn(


O pré-requisito para cursar Programação para Dispositivos Móveis é a disciplina "Programação Orientada a Objetos" (Competência e Habilidades: Compreender conhecimentos teóricos e práticos, acerca do domínio da programação para dispositivos móveis).
RESPOSTA: O pré-requisito para cursar Programação para Dispositivos Móveis é a disciplina "Programação Orientada a Objetos" (Competência e Habilidades: Compreender conhecimentos teóricos e práticos, acerca do domínio da programação para dispositivos móveis).


---
## 10. Limpeza de Memória (Opcional)
**Execute se precisar liberar recursos da GPU**

In [ ]:
def clear_gpu_memory(*args):
    """Libera a memória da GPU."""
    print("\n--- Limpando a memória da GPU ---")
    for arg in args:
        try:
            del arg
        except NameError:
            pass
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        print("✓ Cache da GPU limpo.")

# Descomente para limpar memória
# clear_gpu_memory(llm, embeddings, vectorstore, cross_encoder)

---
## 11. Executar Múltiplas Rodadas de Avaliação
**Execute esta seção para rodar o dataset N vezes e obter estatísticas agregadas**

In [ ]:
# Configurar número de rodadas
NUM_RODADAS = 5  # Altere para quantas rodadas desejar

print(f"Configurado para executar {NUM_RODADAS} rodadas de avaliação")

Configurado para executar 5 rodadas de avaliação


In [ ]:
# Executar múltiplas rodadas
all_rounds_results = []
all_rounds_metrics = []

for rodada in range(1, NUM_RODADAS + 1):
    print(f"\n\n{'#'*80}")
    print(f"RODADA {rodada}/{NUM_RODADAS}")
    print(f"{'#'*80}\n")

    rodada_results = []

    for idx, item in enumerate(evaluation_dataset, 1):
        print(f"\n[Rodada {rodada}] Pergunta {idx}/{len(evaluation_dataset)}")

        generated_answer, retrieved_context = ask(
            item["pergunta"],
            rag_chain,
            retriever,
            cross_encoder,
            verbose=False  # Menos verboso em múltiplas rodadas
        )

        rodada_results.append({
            "rodada": rodada,
            "pergunta": item["pergunta"],
            "resposta_esperada": item["resposta_esperada"],
            "resposta_gerada": generated_answer,
            "contexto_recuperado": retrieved_context
        })

    # Calcular métricas para esta rodada
    gen_ans = [r["resposta_gerada"] for r in rodada_results]
    exp_ans = [r["resposta_esperada"] for r in rodada_results]

    rouge = rouge_metric.compute(predictions=gen_ans, references=exp_ans)
    bleu = bleu_metric.compute(predictions=gen_ans, references=exp_ans)
    bertscore = bertscore_metric.compute(
        predictions=gen_ans, references=exp_ans,
        lang='pt', model_type='distilbert-base-multilingual-cased'
    )

    f1_round = [calculate_f1_token(p, r) for p, r in zip(gen_ans, exp_ans)]

    rodada_metrics = {
        'rodada': rodada,
        'ROUGE-1': rouge['rouge1'],
        'ROUGE-2': rouge['rouge2'],
        'ROUGE-L': rouge['rougeL'],
        'BLEU': bleu['bleu'],
        'BERTScore_F1': sum(bertscore['f1']) / len(bertscore['f1']),
        'F1_Token': sum(f1_round) / len(f1_round),
    }

    all_rounds_results.extend(rodada_results)
    all_rounds_metrics.append(rodada_metrics)

    print(f"\n✓ Rodada {rodada} concluída!")
    print(f"  F1-Token: {rodada_metrics['F1_Token']:.4f}")

print(f"\n\n{'='*80}")
print(f"✓ TODAS AS {NUM_RODADAS} RODADAS CONCLUÍDAS!")
print("="*80)



################################################################################
RODADA 1/5
################################################################################


[Rodada 1] Pergunta 1/10

[DEBUG] Top 4 documentos após re-ranking:
  #1 (score: 7.1133): denominação: Análise e Desenvolvimento de Sistemas. Em 2002, foi autorizada a cr...
  #2 (score: 4.9391): O Instituto Federal do Piauí possui atualmente 20 campi distribuídos do norte ao...
  #3 (score: 4.6784): MINISTÉRIO DA EDUCAÇÃO Secretaria de Educação Profissional e Tecnológica Institu...
  #4 (score: 3.6548): de Redação do Vestibular/Exame Nacional do Ensino Médio (ENEM) em um dos últimos...


/usr/local/lib/python3.12/dist-packages/llama_cpp/llama.py:1242: RuntimeWarning: Detected duplicate leading "<|begin_of_text|>" in prompt, this will likely reduce response quality, consider removing it...
  warnings.warn(


Em 2013, no Campus de Picos.
[Rodada 1] Pergunta 2/10

[DEBUG] Top 4 documentos após re-ranking:
  #1 (score: 4.3135): entre o primeiro e o último período do curso; - Trabalho de Conclusão de Curso (...
  #2 (score: 4.0929): de Projeto Carga horária: 40h/a Módulo: IV Código(s) do Pré-requisito 17 a 23 Co...
  #3 (score: 4.0589): estará sujeito a uma arguição aberta aos membros da banca examinadora e à platei...
  #4 (score: 3.2253): de elaboração de trabalhos acadêmicos do IFPI. Os artigos científicos completos ...


/usr/local/lib/python3.12/dist-packages/llama_cpp/llama.py:1242: RuntimeWarning: Detected duplicate leading "<|begin_of_text|>" in prompt, this will likely reduce response quality, consider removing it...
  warnings.warn(


De acordo com o trecho 4, as modalidades aceitas para a apresentação do TCC são: monografias e artigos científicos completos (com resultados) publicados em eventos regionais, nacionais e internacionais ou em periódicos pertencentes à Lista de Periódicos classificados no Qualis Capes ou na Revista Somma do IFPI. Além disso, os alunos também podem desenvolver como projeto da disciplina de TCC um Relatório Técnico de Software (RTS) ou um Relatório Técnico de Trabalho/Estágio (RTT).
[Rodada 1] Pergunta 3/10

[DEBUG] Top 4 documentos após re-ranking:
  #1 (score: 4.8680): serviços, hardware e também as exportações do segmento. Em 2021, o PIB nacional ...
  #2 (score: 1.2843): da manutenção da modalidade home-office que ainda foi necessária. Para poder con...
  #3 (score: 1.1567): % podem ser classificadas como micro e pequenas empresas, segundo análise realiz...
  #4 (score: 0.4441): que atuem na forma de consultoria em Sistemas de Informação, avaliando e selecio...


...

A nota final do Trabalho de Conclusão de Curso (TCC) é calculada com base na média ponderada das notas dos membros da banca examinadora. A fórmula utilizada para calcular a média é:

Média = (50 x N1 + 25 x N2 + 25 x N3) / 100

Onde:
- N1 é a nota do orientador (e/ou co-orientador);
- N2 e N3 são as notas dos demais membros da banca examinadora.

Essa fórmula atribui uma ponderação de 50% para a nota do orientador, e 25% cada para as notas dos outros dois membros da banca.
[Rodada 5] Pergunta 9/10

[DEBUG] Top 4 documentos após re-ranking:
  #1 (score: 0.9402): Pré-requisito: Não possui Competência e Habilidades: Apresentação de temas atuai...
  #2 (score: -0.1022): Pré-requisito: 08. Introdução a Administração Competência e Habilidades: Compree...
  #3 (score: -1.4401): da Educação. Base Nacional Comum Curricular. Brasília, 2018. Computação - Comple...
  #4 (score: -1.8217): básica: 1. TANEMBAUM, Andrew S et al. Redes de Computadores. 6ª. Edição. Bookman...


/usr/local/lib/python3.12/dist-packages/llama_cpp/llama.py:1242: RuntimeWarning: Detected duplicate leading "<|begin_of_text|>" in prompt, this will likely reduce response quality, consider removing it...
  warnings.warn(


A disciplina "Introdução a Computação" (03) é o pré-requisito necessário para cursar Redes de Computadores.
[Rodada 5] Pergunta 10/10

[DEBUG] Top 4 documentos após re-ranking:
  #1 (score: 2.1200): Ciência e Tecnologia. O IFPI possui 113 (cento e treze) anos, tendo origem na Es...
  #2 (score: 2.0640): de qualificação de mão de obra no processo de industrialização brasileira. Nessa...
  #3 (score: 2.0127): Instituição para o curso superior de Tecnologia em Informática. Em 2001 e 2002, ...
  #4 (score: 1.1193): de opções, vieram também os cursos técnicos da área de serviços, como os de Cont...


/usr/local/lib/python3.12/dist-packages/llama_cpp/llama.py:1242: RuntimeWarning: Detected duplicate leading "<|begin_of_text|>" in prompt, this will likely reduce response quality, consider removing it...
  warnings.warn(


A instituição, que hoje é o IFPI, foi efetivamente transformada em Centro Federal de Educação Tecnológica do Piauí (CEFET-PI) em 1999.
✓ Rodada 5 concluída!
  F1-Token: 0.5408


✓ TODAS AS 5 RODADAS CONCLUÍDAS!


In [ ]:
# Análise agregada das rodadas
df_all_rounds = pd.DataFrame(all_rounds_results)
df_metrics_rounds = pd.DataFrame(all_rounds_metrics)

print("\nESTATÍSTICAS AGREGADAS DE TODAS AS RODADAS:\n")
# Display describe() output as a DataFrame
display(df_metrics_rounds.describe().round(4))

print("\n\nMÉDIAS FINAIS (todas as rodadas):\n")
final_averages_data = {}
for col in df_metrics_rounds.columns:
    if col != 'rodada':
        mean_val = df_metrics_rounds[col].mean()
        std_val = df_metrics_rounds[col].std()
        final_averages_data[col] = [mean_val, std_val]

df_final_averages = pd.DataFrame(final_averages_data, index=['Média', 'Desvio Padrão'])
display(df_final_averages.round(4))


# Calcular e exibir médias por rodada
print("\n\nMÉTRICAS MÉDIAS POR RODADA:\n")
# Ensure the relevant columns exist before grouping
metrics_cols = ['ROUGE-1', 'ROUGE-2', 'ROUGE-L', 'BLEU', 'BERTScore_F1', 'F1_Token']
df_metrics_rounds_grouped = df_metrics_rounds.groupby('rodada')[metrics_cols].mean()
display(df_metrics_rounds_grouped.round(4))


ESTATÍSTICAS AGREGADAS DE TODAS AS RODADAS:



,rodada,ROUGE-1,ROUGE-2,ROUGE-L,BLEU,BERTScore_F1,F1_Token
count,5.0000,5.0000,5.0000,5.0000,5.0000,5.0000,5.0000
mean,3.0000,0.5856,0.4312,0.4843,0.2248,0.9147,0.5081
std,1.5811,0.0163,0.0159,0.0228,0.0162,0.0036,0.0203
min,1.0000,0.5658,0.4155,0.4630,0.2066,0.9097,0.4873
25%,2.0000,0.5780,0.4216,0.4753,0.2142,0.9133,0.4956
50%,3.0000,0.5868,0.4295,0.4765,0.2237,0.9151,0.5083
75%,4.0000,0.5871,0.4325,0.4838,0.2308,0.9158,0.5084
max,5.0000,0.6103,0.4571,0.5229,0.2486,0.9196,0.5408




MÉDIAS FINAIS (todas as rodadas):



,ROUGE-1,ROUGE-2,ROUGE-L,BLEU,BERTScore_F1,F1_Token
Média,0.5856,0.4312,0.4843,0.2248,0.9147,0.5081
Desvio Padrão,0.0163,0.0159,0.0228,0.0162,0.0036,0.0203




MÉTRICAS MÉDIAS POR RODADA:



,ROUGE-1,ROUGE-2,ROUGE-L,BLEU,BERTScore_F1,F1_Token
rodada,,,,,,
1,0.5658,0.4155,0.4630,0.2066,0.9097,0.4873
2,0.5868,0.4325,0.4838,0.2308,0.9158,0.5083
3,0.5871,0.4295,0.4753,0.2237,0.9151,0.4956
4,0.5780,0.4216,0.4765,0.2142,0.9133,0.5084
5,0.6103,0.4571,0.5229,0.2486,0.9196,0.5408


In [ ]:
# Salvar resultados de múltiplas rodadas
df_all_rounds.to_csv("resultados_multiplas_rodadas.csv", index=False)
df_metrics_rounds.to_csv("metricas_multiplas_rodadas.csv", index=False)

print("✓ Resultados de múltiplas rodadas salvos!")
print("  - resultados_multiplas_rodadas.csv")
print("  - metricas_multiplas_rodadas.csv")

✓ Resultados de múltiplas rodadas salvos!
  - resultados_multiplas_rodadas.csv
  - metricas_multiplas_rodadas.csv


---
## 📝 Instruções de Uso

### Fluxo Recomendado:

1. **Primeira Execução (Setup Completo):**
   - Execute seções 1 e 2 (Imports + Construção do Índice) - APENAS 1 VEZ
   - Execute seção 3 (Carregamento do Sistema RAG)
   - Execute seção 4 (Funções)

2. **Avaliações Subsequentes:**
   - Pule a seção 2 (índice já existe)
   - Execute seção 3 (Carregar sistema)
   - Execute seção 4 (Funções)
   - Configure dataset na seção 5
   - Execute seção 6 (Processar perguntas)
   - Execute seção 7 (Calcular métricas)

3. **Para Múltiplas Rodadas:**
   - Configure NUM_RODADAS na seção 11
   - Execute toda a seção 11 para análise estatística

4. **Teste Rápido:**
   - Use seção 9 para testar perguntas individuais sem avaliação formal

### Dicas:
- Os modelos são carregados **uma vez** e reutilizados
- Adicione quantas perguntas quiser na seção 5
- As métricas são calculadas de forma **independente** das perguntas
- Use a seção 10 se precisar liberar memória GPU entre execuções